In [ ]:
# --------------------------------------------------------
# RP-FEM: Relational Prior Knowledge Graphs for Detection and Instance Segmentation
# Copyright (c) 2023
# Licensed under The MIT License [see LICENSE for details]
# --------------------------------------------------------

import os
import json
import math
import numpy as np

from tqdm import tqdm

In [ ]:
ROOT_VG = "path/to/VisualGenome/"
with open(ROOT_VG + 'objects.json') as in1:
    objects_file = json.load(in1)

with open(ROOT_VG + 'image_data.json') as in2:
    image_data = json.load(in2)
    
with open('SYNSET_TO_NAME_MAPPER.json') as in3:
    SYNSET_TO_NAMES_MAPPER = json.load(in3)

with open('COCO_TO_VG_SYNSET_MAPPER.json') as in4:
    COCO_TO_VG_SYNSET_MAPPER = json.load(in4)
    
with open('COCO_CLASSES_IN_VG_COUNT.json') as in5:
    COCO_CLASSES_IN_VG_COUNT = json.load(in5)

all_ids = []
all_files = os.listdir(ROOT_VG + "VG_100K")
for i in all_files:
    all_ids.append(i.split(".")[0])
    
new_image_data = {}
for all_data in image_data:
    new_image_data[all_data['image_id']] = all_data

In [ ]:
COCO_TO_VG_SYNSET_MAPPER_INVERSE = {}
for p in COCO_TO_VG_SYNSET_MAPPER:
    for m in COCO_TO_VG_SYNSET_MAPPER[p]:
        if m not in COCO_TO_VG_SYNSET_MAPPER_INVERSE:
            COCO_TO_VG_SYNSET_MAPPER_INVERSE[m] = [p]
        else:
            COCO_TO_VG_SYNSET_MAPPER_INVERSE[m].append(p)

In [ ]:
abs_dict_dict = {}
for c in COCO_TO_VG_SYNSET_MAPPER:
    inner_dict = {} 
    for cc in COCO_TO_VG_SYNSET_MAPPER:
        
        # {L, C, R, T, B} stand for {“left", “center", “right", “top", “bottom"}
        # as in https://arxiv.org/pdf/2206.00481.pdf
        inner_dict[cc] = []
    abs_dict_dict[c] = inner_dict


In [ ]:
def euc_dist(x1, y1, x2, y2):
    return math.sqrt((x1 - x2)**2 + (y1 - y2)**2)

In [ ]:
relevant_classes = [j for sub in list(COCO_TO_VG_SYNSET_MAPPER.values()) for j in sub]
for c_i, image in tqdm(enumerate(objects_file), total=len(objects_file)):
    if str(image["image_id"]) not in all_ids:
        continue
    this_image_data = new_image_data[image["image_id"]]
    height, width = this_image_data['height'], this_image_data['width']
    for object_i in image['objects']:

        valid_names_i = []
        stripped_names_i = [synset_i.split(".")[0].replace('_', ' ') for synset_i in object_i["synsets"]]
        new_stripped_names_i = []

        for name in stripped_names_i:
            for k in COCO_TO_VG_SYNSET_MAPPER:
                key_list = COCO_TO_VG_SYNSET_MAPPER[k]
                if name in key_list:
                    if name not in new_stripped_names_i:
                        new_stripped_names_i.append(name)
        if new_stripped_names_i == []:
            continue
            
        my_x_center = ((object_i['w'] / 2) + object_i['x']) / width
        my_y_center = ((object_i['h'] / 2) + object_i['y']) / height
        for c, object_j in enumerate(image['objects']):
            if object_i['object_id'] == object_j['object_id']:
                continue
            
            valid_names_j = []
            stripped_names_j = [synset_j.split(".")[0].replace('_', ' ') for synset_j in object_j["synsets"]]
            new_stripped_names_j = []
            for name in stripped_names_j:
                for k in COCO_TO_VG_SYNSET_MAPPER:
                    key_list = COCO_TO_VG_SYNSET_MAPPER[k]
                    if name in key_list:
                        if name not in new_stripped_names_j:
                            new_stripped_names_j.append(name)
            if new_stripped_names_j == []:
                continue
                            
            their_x_center = ((object_j['w'] / 2) + object_j['x']) / width
            their_y_center = ((object_j['h'] / 2) + object_j['y']) / height
            
            dist = euc_dist(my_x_center, my_y_center, their_x_center, their_y_center)

            for ss_i in new_stripped_names_i:
                inverse_list_ss_i = COCO_TO_VG_SYNSET_MAPPER_INVERSE[ss_i]
                for all_inverse_ssi in inverse_list_ss_i:
                    for ss_j in new_stripped_names_j:
                        inverse_list_ss_j = COCO_TO_VG_SYNSET_MAPPER_INVERSE[ss_j]
                        for all_inverse_ssj in inverse_list_ss_j:
                            
                            # Normalize distance with bounding box sizes
                            object_i_area = object_i['w'] * object_i['h']
                            object_j_area = object_j['w'] * object_j['h']
                            if abs(object_i_area - object_j_area) == 0:
                                normalization_term = 1
                            else:
                                normalization_term = (object_i_area + object_j_area) / abs(object_i_area - object_j_area)

                            # Distance relation
                            dist /= normalization_term
                            abs_dict_dict[all_inverse_ssi][all_inverse_ssj].append(dist)


In [ ]:
# Compute mean and standard deviation
mean_std_dict = {}
for class_ratios in list(abs_dict_dict.keys()):
    mean_std_dict[class_ratios] = {}

    for class_ratios2 in list(abs_dict_dict.keys()):
        class_ratios_list = abs_dict_dict[class_ratios][class_ratios2]
        mean = np.mean(class_ratios_list)
        std = np.std(class_ratios_list)
        if math.isnan(mean) or math.isnan(std):
            mean_std_dict[class_ratios][class_ratios2] = [0, 0]
        else:
            mean_std_dict[class_ratios][class_ratios2] = [mean, std]

In [ ]:
with open("relative-distance.json", "w") as out1:
    json.dump(mean_std_dict, out1)
    